In [25]:
import pandas as pd
import numpy as np
import json
from dateutil import rrule, parser
from dataclasses import dataclass, field
from datetime import date, datetime, timedelta
from intervaltree import Interval, IntervalTree
#from generate_operators_availability import generate_operator_availability

# Definition of the different classes
@dataclass
class Operator:
    name: str
    skills: list
    holidays: IntervalTree
    availabilty: IntervalTree = field(default_factory=IntervalTree)
    shift: list = field(default_factory=list)

@dataclass
class Step:
    name: str
    step_number: int
    duration: timedelta
    capacity: int
    log: pd.DataFrame
    
# Load data from the JSON configuration file
with open("SimulatorInputs.json", "r") as file:
    data = json.load(file)

# Extract the simulation parameters
json_simulation_start: str = data['SimulationParameters']['simulation_start']
simulation_start: datetime = datetime.fromisoformat(json_simulation_start)

json_simeulation_end: str = data['SimulationParameters']['simulation_end']
simulation_end: datetime = datetime.fromisoformat(json_simeulation_end)

modules_planned_deliveries: str = data["ComponentArrivalTimes"]["bare modules"]

total_modules_number: int = 0
for delivery in modules_planned_deliveries:
    total_modules_number += modules_planned_deliveries[delivery]

# Initialize instances of the operator class
# All the operators are stored in the operators list
operators = []

# Initialize the CERN holidays to the good format
CERN_holidays_series = pd.Series(data["CERNHolidays"])
CERN_holidays_series[:] = CERN_holidays_series.apply(lambda x: [datetime.fromisoformat(date) for date in x])

for id, operator in enumerate(data["Operators"]):
    # Loads the operator's individual holidays
    individual_holidays = pd.Series(data["OperatorHolidays"][f"Operator{id + 1}"])
    individual_holidays[:] = individual_holidays.apply(lambda x: [datetime.fromisoformat(date) for date in x])

    # Merges the individual holidays with the CERN holidays
    operator_holidays = pd.concat([individual_holidays, CERN_holidays_series])
    operator_holidays[:] = operator_holidays.apply(lambda x: Interval(x[0], x[1], "holiday"))

    # Converts the pandas series to a list and then to an interval tree
    operator_holidays_list = operator_holidays.tolist()
    operator_holidays_tree = IntervalTree(operator_holidays_list)

    globals()[f"Operator{id + 1}"] = Operator(name=f"Operator{id + 1}", skills=operator, holidays=operator_holidays_tree)
    operators.append(globals()[f"Operator{id + 1}"])


# Initialization of the Step class instances, they are stored in the "Chronologically_Ordered_Steps" dict, the member log is a dataframe that will contain 
# the entry and exit dates of the modules at each step

Chronologically_Ordered_Steps = {}
for id, step in enumerate(data["StagesAndSteps"]):
    step_name = step["Step"].replace("-", "_").replace(" ", "_")
    globals()[step_name] = Step(name=step["Step"], step_number=id, duration=step["Duration"], capacity= step["Capacity"],log=pd.DataFrame(columns=["Entry_Date", "Exit_Date"]))
    Chronologically_Ordered_Steps |= {step["Step"]: globals()[step_name]}

time: datetime = simulation_start
finished_modules_count: int = 0 #Might be a duplicate

# Initialization of the list of the differents tasks and steps to be done
tasks_to_do: list = [task['Step'] for task in data["StagesAndSteps"]]

# Initialization of the dataframe that will contain the assignments of the operators
operators_assignments: pd.DataFrame = pd.DataFrame(columns=[tasks_to_do], index=[simulation_start])


In [26]:
# Load the state of production csv data into each step
state_of_production = pd.read_csv("inventory.csv", sep=";", dtype={"Quantity":'Int64'}, index_col=0, parse_dates=["Launching Time"])
state_of_production.fillna({"Quantity": 0}, inplace=True)

# This piece of code fills by default the missing values of the Ready components launching time
# to simulation_start_time - duration of the task to simulate the fact that they are just ready to 
# be moved to the next step when the simulation is launched. 
for index, row in state_of_production.iterrows():   # I prefer to work with literal values of Index and Columns rather than the integer values for more reliability and legibility
    if row["Quantity"] > 0:     # We only care about the Steps where there are modules
        for task in iter(Chronologically_Ordered_Steps):
            if task in index:   # Looks for the task associated with the row TODO : suppress this loop by implementing yet another lookup table
                if pd.isna(state_of_production.loc[index, "Launching Time"]):
                    Time_ready_by_simulation_start = simulation_start - timedelta(minutes=Chronologically_Ordered_Steps[task].duration)   # In the last part we extract the duration associated with the task
                    state_of_production.loc[index, "Launching Time"] = Time_ready_by_simulation_start

# Now we fill all those initial data into the log attribute of each step
for Step in Chronologically_Ordered_Steps.values():
    row_Ready = state_of_production.loc[Step.name + " Ready"]
    row_WIP = state_of_production.loc[Step.name + " WIP"]
    df_Ready = pd.DataFrame([[row_Ready.loc["Launching Time"],pd.NaT] for _ in range(row_Ready.loc["Quantity"])],columns=["Entry_Date", "Exit_Date"])
    df_WIP = pd.DataFrame([[row_WIP.loc["Launching Time"],pd.NaT] for _ in range(row_WIP.loc["Quantity"])],columns=["Entry_Date", "Exit_Date"])
    Step.log = pd.concat([Step.log, df_Ready, df_WIP], ignore_index=True)


C:\Users\Martin\AppData\Local\Temp\ipykernel_29720\3239257992.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Step.log = pd.concat([Step.log, df_Ready, df_WIP], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_29720\3239257992.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Step.log = pd.concat([Step.log, df_Ready, df_WIP], ignore_index=True)
C:\Users\Martin\AppData\Local\Temp\ipykernel_29720\3239257992.py:22: FutureWarning: The behavior of DataFrame concatenation with e

In [46]:
Steps = list(reversed(Chronologically_Ordered_Steps.values()))[:-1]
Previous_Steps = list(reversed(Chronologically_Ordered_Steps.values()))[2:]

In [47]:
type(Previous_Steps)

list

In [ ]:
steps_to_do = []
for Step,Previous_Step in zip(Steps, Previous_Steps):
    time_array = np.full_like(Previous_Step.log.loc[:,"Entry_Date"], time)
    time_spent_in_task = time_array - Previous_Step.log.loc[:,"Entry_Date"]
    duration_delta = timedelta(minutes=Previous_Step.duration) 
    if sum(time_spent_in_task >= duration_delta) >= Step.capacity:
        steps_to_do.append(Step)

steps_to_do

[Step(name='FA - visual inspection of module', step_number=9, duration=10, capacity=1, log=Empty DataFrame
 Columns: [Entry_Date, Exit_Date]
 Index: []),
 Step(name='Rec - Metrology and Visual inspection of bare modules or flexes', step_number=3, duration=20, capacity=1, log=Empty DataFrame
 Columns: [Entry_Date, Exit_Date]
 Index: []),
 Step(name='Rec - PDB checks of bare modules and flexes', step_number=2, duration=10, capacity=20, log=Empty DataFrame
 Columns: [Entry_Date, Exit_Date]
 Index: [])]

In [ ]:
steps_to_do